In [1]:
import json
import os
from utils import Evaluation, CriteraFeedback, get_structured_feedback, aggregate_feedback, pretty_format_evaluation
from dotenv import load_dotenv
from openai import OpenAI
import instructor

In [2]:
load_dotenv()

openai_client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

instructor_client = instructor.from_openai(openai_client)

In [3]:
user_ids = ["user1", "user2"]

Store Aggregated feedback for each user

In [4]:
for user_id in user_ids:
	with open(f"{user_id}-messages.json", "r") as f:
		chat_history = json.load(f)
	
	print(f"Aggregating feedback for user {user_id}")
	
	# Aggregate feedback from chat history for current user
	feedbacks = get_structured_feedback(instructor_client, chat_history)
	aggregated_feedback: Evaluation = aggregate_feedback(feedbacks, openai_client)

	# Store User's aggregated feedback
	with open(f"aggregated-feedback--user-{user_id}.json", "w") as f:
		f.write(aggregated_feedback.model_dump_json(indent=2))
	
	print(f"Stored aggregated feedback for user {user_id}")

Aggregating feedback for user user1


/Users/13point5/Library/Caches/pypoetry/virtualenvs/classroom-lm-poc-GKspPBel-py3.11/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/13point5/Library/Caches/pypoetry/virtualenvs/classroom-lm-poc-GKspPBel-py3.11/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/13point5/Library/Caches/pypoetry/virtualenvs/classroom-lm-poc-GKspPBel-py3.11/lib/python3.11/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/13point5/Library/Caches/pypoetry/virtualenvs/classroom-lm-poc-GKspPBel-py3.11/lib/python3.11/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/Users/13point5/Library/Cach

Stored aggregated feedback for user user1
Aggregating feedback for user user2
Stored aggregated feedback for user user2


In [5]:
student_aggregated_feedbacks = []

for user_id in user_ids:
	with open(f"aggregated-feedback--user-{user_id}.json", "r") as f:
		student_aggregated_feedbacks.append(Evaluation(**json.load(f)))

student_aggregated_feedbacks

[Evaluation(introduction=CriteraFeedback(strengths=['Effectively framing the argument in the introduction and conclusion', 'Clear and compelling stance on environmental conservation', 'Clear thesis statement'], weaknesses=['Lack of engaging opening statements', 'Lacks a clear thesis statement'], suggestions=["Consider using a more captivating opening to grab the reader's attention", 'Include a clear thesis statement at the end of the introduction paragraph']), structure=CriteraFeedback(strengths=['Clear and logical organization of the essay', 'Organized paragraphs'], weaknesses=['Counterargument section could be developed further', 'Lack of transition sentences between paragraphs'], suggestions=['Develop the counterargument with more details or statistics to strengthen the position', 'Include transition sentences to improve flow']), argumentation=CriteraFeedback(strengths=['Use of examples to solidify the argument', 'Strong and clear arguments'], weaknesses=[], suggestions=['Consider u

In [6]:
classroom_aggregated_feedback = aggregate_feedback(openai_client=openai_client, feedbacks=student_aggregated_feedbacks)

with open("classroom-aggregated-feedback.json", "w") as f:
	f.write(classroom_aggregated_feedback.model_dump_json(indent=2))

In [7]:
classroom_aggregated_feedback

Evaluation(introduction=CriteraFeedback(strengths=['Effectively framing the argument in the introduction and conclusion', 'Clear and compelling stance on environmental conservation', 'Relatable and engaging language'], weaknesses=['Lack of engaging opening statements', 'Lacks a clear thesis statement'], suggestions=["Consider using a more captivating opening to grab the reader's attention", 'Add more factual information and statistics', 'Maintain a balance between conversational and formal tone']), structure=CriteraFeedback(strengths=['Clear and logical organization of the essay', 'Organized paragraphs'], weaknesses=['Counterargument section could be developed further', 'Lack of transition sentences between paragraphs'], suggestions=['Develop the counterargument with more details or statistics to strengthen the position', 'Include transition sentences to improve flow', 'Structure the essay with clear introductions, body paragraphs, and conclusions']), argumentation=CriteraFeedback(stre

In [8]:
evaluation_str = pretty_format_evaluation(classroom_aggregated_feedback)

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage

In [10]:
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "You are a Lesson Plan Generating tool for a grade 8 English classroom."
                "You will be given a query by the teacher along with an aggregate evaluation"
                "of all the students based on their interaction with the Essay Feedback provider tool"
                "Use both pieces of information to personalise your outputs. Use markdown."
				"Only provide the lesson plan, do not regurgitate the evaluation.\n\n"
				f"Class Evaluation: {evaluation_str}"
            )
        )
    ]
)

model = ChatOpenAI()

chain = prompt | model

In [11]:
query = "world war 2"

result = chain.invoke({
	"messages": [
		HumanMessage(
			content=(
				f"Generate a lesson plan to teach this topic: {query}"
			)
		)
	]
})

In [12]:
print(result.content)

# Lesson Plan: Improving Essay Writing Skills

## Objective:
- Students will be able to enhance their essay writing skills by incorporating engaging opening statements, developing counterarguments with evidence, and crafting impactful closing statements.

## Duration:
- 2 class periods (45 minutes each)

## Materials:
- Copies of the students' previous essays
- Whiteboard and markers
- Sample essays with strong opening, counterargument, and closing statements

## Lesson Outline:

### Introduction (10 minutes):
1. Begin by discussing the importance of engaging opening statements in an essay.
2. Show examples of captivating opening statements from sample essays.
3. Ask students to share their previous opening statements and discuss ways to make them more engaging.

### Body (60 minutes):
1. Focus on developing counterarguments with evidence.
   - Explain the significance of addressing opposing viewpoints in an essay.
   - Provide examples of well-developed counterarguments with statistic